![X-Ray - Image CC0 - pexels.com](./img/pexels-anna-shvets-4226258.jpg "X-Ray")

Chest X-Ray encoder (CXRE)
==========================

## Description

## Getting data

In [1]:
import download_data

In [4]:
# Download the data
## Fill the token

token = ... # Fill the current line

download_data.download_data(token=token, folder="./data")

Unziping downloaded file


BadZipFile: File is not a zip file

## Data description